In [ ]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from tuning_curves_functions import get_tc, get_odd_firing_idx, linearize

import info.R063d3_info as r063d3
import info.R066d4_info as r066d4

In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'

In [ ]:
info = r063d3

In [ ]:
print(info.session_id)
pos = info.get_pos(info.pxl_to_cm)

t_start = info.task_times['phase2'][0]
t_stop = info.task_times['phase2'][1]

t_start_idx = vdm.find_nearest_idx(pos['time'], t_start)
t_end_idx = vdm.find_nearest_idx(pos['time'], t_stop)

sliced_pos = dict()
sliced_pos['x'] = pos['x'][t_start_idx:t_end_idx]
sliced_pos['y'] = pos['y'][t_start_idx:t_end_idx]
sliced_pos['time'] = pos['time'][t_start_idx:t_end_idx]

linear, zone = linearize(info, pos)

spikes = info.get_spikes()

tc = get_tc(info, sliced_pos, pickle_filepath)

# sort_idx = vdm.get_sort_idx(tc['u'])
# odd_firing_idx = get_odd_firing_idx(tc['u'])

# ordered_spikes = spikes['time'][sort_idx]

In [ ]:
linear = linear['u']
tc = np.array(tc['u'])

dt = np.median(np.diff(linear['time']))
edges = np.hstack((linear['time']-(dt/2), linear['time'][-1]))
subsample = 6
edges = edges[::subsample]
counts = vdm.get_counts(spikes['time'], edges)

In [ ]:
plt.pcolormesh(counts[:,:100])
plt.colorbar()
plt.show()

In [ ]:
centers = edges[:-1] + np.median(np.diff(edges))/2
prob = vdm.bayesian_prob(counts, tc, centers)

In [ ]:
plt.pcolormesh(prob[200::-1])
plt.colorbar()
plt.show()

In [ ]:
decoded_position = vdm.decode_location(prob, linear)

decoded = dict()
decoded['time'] = centers
decoded['position'] = decoded_position

actual_idx = vdm.find_nearest_indices(linear['time'], decoded['time'])
actual_location = linear['position'][actual_idx]

# decoded['position'][np.isnan(decoded['position'])] = 0
decode_error = np.abs(actual_location - decoded['position'])
print(np.mean(decode_error))

In [ ]:
plt.plot(decoded['time'], decoded['position'], 'b.')
plt.plot(linear['time'], linear['position'], 'r.')
plt.show()

In [ ]:
def find_jumps(decoded, min_idx_length=3, max_idx_jump=20):
    """Finds intervals of decoded position that are within jump limits.
    
    Parameters
    ----------
    decoded : np.array
        Estimate of decoded location (floats) for each time bin.
    min_idx_length : int
        Minimum number of bins to be considered a sequence.
    max_idx_jump : int
        Maximum number of bins to break a sequence.
    
    Returns
    -------
    sequences : dict
        With time (tuple of floats), index (tuple of ints) as keys.
        Where start, stop are [0], [1] of each tuple.
    
    """
    previous_position = -100
    start_idx = []
    sequences = dict(time=[], index=[])

    for idx, position in enumerate(decoded['position']):
        this_jump = np.abs(position - previous_position)
        if this_jump <= max_idx_jump:
            if len(start_idx) < 1:
                start_idx.append(idx-1)
            else:
                if ((idx-1) - start_idx[0]) >= min_idx_length:
                    sequences['time'].append((decoded['time'][start_idx[0]], decoded['time'][idx-1]))
                    sequences['index'].append((start_idx[0], idx-1))
                    start_idx = []
        previous_position = position
    
    return sequences
        

In [ ]:
print(sequences['time'][:10], sequences['index'][:10])

In [ ]:
plt.plot(linear['time'], linear['position'], 'r.')
for seq in sequences['index']:
    plt.plot(decoded['time'][seq[0]:seq[1]], decoded['position'][seq[0]:seq[1]], 'b.')

plt.show()

In [ ]:
avg_error = []
decoded['position'][np.isnan(decoded['position'])] = 0
for seq in sequences['index']:
    decode_error = np.abs(actual_location[seq[0]:seq[1]] - decoded['position'][seq[0]:seq[1]])
    avg_error.append(np.mean(decode_error))
print(np.mean(avg_error))

In [ ]:
dt = np.median(np.diff(linear['u']['time']))

edges = np.hstack(((linear['u']['time']-dt)/2, linear['u']['time'][-1]))
subsample = 6
edges = edges[np.arange(1, len(edges)+1, 6)]

centers = (edges[:-1] + dt)/2

In [ ]:
position_z = linear['u']
num_bins = 101

linear_start = np.min(position_z['time'])
linear_stop = np.max(position_z['time'])
edges = np.linspace(linear_start, linear_stop, num=num_bins)
centers = np.array((edges[1:] + edges[:-1]) / 2.)

# occupancy = np.zeros(len(centers))

In [ ]:
intervals = dict()
intervals['start'] = edges[:-2]
intervals['stop'] = edges[1:-1]

counts = vdm.spike_counts(spikes['time'], intervals)

In [ ]:
print(np.min(counts), np.max(counts))

In [ ]:
print(np.shape(counts))

In [ ]:
print(intervals['start'][0:5], intervals['stop'][0:5])

In [ ]:
plt.plot(counts)
plt.show()

# From matlab

In [ ]:
import scipy.io as sio
loading_decoder = sio.loadmat('C:\\Users\\Emily\\Desktop\\_decoding.mat')

In [ ]:
decode = dict(time=[])
decode['spikes'] = loading_decoder['pyspikes'][0]
decode['ztime'] = loading_decoder['pyztime'][0]
decode['zdata'] = loading_decoder['pyzdata'][0]
decode['tc'] = loading_decoder['pytc']

In [ ]:
print(np.shape(decode['tc']))

In [ ]:
dt = np.median(np.diff(decode['ztime']))

edges = np.hstack((decode['ztime']-(dt/2), decode['ztime'][-1]))
subsample = 6
edges = edges[::subsample]

In [ ]:
print(len(edges), np.min(edges), np.max(edges))

In [ ]:
this_dt = np.median(np.diff(edges))

In [ ]:
print(this_dt, dt)

In [ ]:
decode['ztime'][:10]

In [ ]:
print(len(centers), np.min(centers), np.max(centers))

In [ ]:
from scipy import signal
gaussian_std = 0.02 / this_dt
gaussian_window = 1.0 / this_dt

gaussian_filter = signal.gaussian(gaussian_window, gaussian_std)
gaussian_filter /= np.sum(gaussian_filter)

In [ ]:
q = np.zeros((int(len(decode['spikes'])), int(len(edges)-1)))
for idx, neuron_spikes in enumerate(decode['spikes']):
    q[idx] = np.histogram(neuron_spikes, bins=edges)[0]
    if gaussian_std > this_dt:
        q[idx] = np.convolve(q[idx], gaussian_filter, mode='same')

In [ ]:
plt.plot(gaussian_filter)
plt.show()

In [ ]:
plt.pcolormesh(q[:,:100])
plt.colorbar()
plt.show()

In [ ]:
print(np.shape(np.histogram(neuron_spikes, bins=edges)[0]))

In [ ]:
print(linear['u'].keys())

In [ ]:
def get_counts(spikes, edges, gaussian_std=0.02, gaussian_window=1.0):
    """Finds the number of spikes in each bin.
    
    Parameters
    ----------
    spikes : np.array
        Where each inner array contains the spike times (floats) for an individual neuron.
    edges : np.array
        Bin edges for computing spike counts.
    gaussian_std : float
        Standard deviation for gaussian filter. Default set to 0.02. Normalized by bin size (dt).
        Only uses filter if this value is greater than dt.
    gaussian_window : float
        Window for gaussian filter. Default set to 1.0. Normalized by bin size (dt).
        Only uses filter if gaussian_std is greater than dt.
    
    Returns
    -------
    counts : np.array
        Where each inner array is the number of spikes (int) in each bin for an individual neuron. 
    
    """
    dt = np.median(np.diff(edges))
    
    apply_filter = False
    gaussian_std /= dt
    gaussian_window /= dt
    
    if gaussian_std > dt:
        apply_filter = True
    
    if apply_filter:
        gaussian_filter = signal.gaussian(gaussian_window, gaussian_std)
        gaussian_filter /= np.sum(gaussian_filter)
        
    counts = np.zeros((int(len(spikes)), int(len(edges)-1)))
    for idx, neuron_spikes in enumerate(spikes):
        counts[idx] = np.histogram(neuron_spikes, bins=edges)[0]
        if apply_filter:
            counts[idx] = np.convolve(counts[idx], gaussian_filter, mode='same')
    return counts

In [ ]:
# linear = dict()
# linear['position'] = decode['zdata']
# linear['time'] = decode['ztime']
# spikes = dict()
# spikes['time'] = decode['spikes']
# tc = decode['tc']

linear = linear['u']
tc = np.array(tc['u'])

dt = np.median(np.diff(linear['time']))
edges = np.hstack((linear['time']-(dt/2), linear['time'][-1]))
subsample = 6
edges = edges[::subsample]
counts = get_counts(spikes['time'], edges)

In [ ]:
plt.pcolormesh(counts[:,:100])
plt.colorbar()
plt.show()

In [ ]:
def bayesian_prob(counts, tuning_curves, centers, min_neurons=1, min_spikes=1):
    """Computes the bayesian probability of location based on spike counts.
    
    Parameters
    ----------
    counts : np.array
        Where each inner array is the number of spikes (int) in each bin for an individual neuron. 
    tuning_curves : np.array
        Where each inner array is the tuning curve (floats) for an individual neuron. 
    centers : np.array
        Bin centers from computing spike counts, based on edges given to get_counts().
    min_neurons : int
        Mininum number of neurons active in a given bin. Default is 1.
    min_spikes : int
        Mininum number of spikes in a given bin. Default is 1.
    
    Returns
    -------
    prob : np.array
        Where each inner array is the probability (floats) for an individual neuron by location bins. 
    
    Notes
    -----
    If a bin does not meet the min_neuron/min_spikes requirement, that bin's probability 
    is set to nan. To convert it to 0s instead, use : prob[np.isnan(prob)] = 0 on the output.
    
    """
    length = np.shape(counts)[1]
    num_bins = np.shape(tuning_curves)[1]
    bin_size = np.median(np.diff(centers))
    
    prob = np.empty((length, num_bins)) * np.nan
    for idx in range(num_bins):
        # What does tempprod represent?
        tempprod = np.nansum(np.log(tuning_curves[:, idx][..., np.newaxis] ** counts), axis=0)

        # What does tempsum represent?
        tempsum = np.exp(-bin_size * np.nansum(tuning_curves[:, idx]))

        prob[:, idx] = np.exp(tempprod) * tempsum * (1/num_bins)


    prob /= np.sum(prob, axis=1)[..., np.newaxis]
    
    num_active_neurons = np.sum(counts > min_spikes, axis=0)
    prob[num_active_neurons < min_neurons] = np.nan
    return prob


In [ ]:
centers = edges[:-1] + np.median(np.diff(edges))/2
prob = bayesian_prob(counts, tc, centers)

plt.pcolormesh(prob[200::-1])
plt.colorbar()
plt.show()

In [ ]:
def decode_location(prob, linear):
    """Finds the decoded location based on a linear (1D) trajectory.
    
    Parameters
    ----------
    prob : np.array
        Where each inner array is the probability (floats) for an individual neuron by location bins.
    linear : dict
        With position (floats), time (floats) as keys.
    
    Returns
    -------
    decoded : np.array
        Estimate of decoded location (floats) for each time bin.
    
    """
    max_decoded_idx = np.argmax(prob, axis=1)
    decoded = max_decoded_idx * (np.max(linear['position'])-np.min(linear['position'])) / (np.shape(prob)[1]-1)
    decoded += np.min(linear['position'])

    nan_idx = np.sum(np.isnan(prob), axis=1) == (np.shape(prob)[1]-1)
    decoded[nan_idx] = np.nan
    
    return decoded

In [ ]:
def find_nearest_indices(array, vals):
    """Finds nearest index in array to value.
    
    Parameters
    ----------
    array : np.array
        This is the array you wish to index into.
    vals : np.array
        This is the array that you are getting your indices from.
    
    Returns
    -------
    Indices into array that is closest to vals.
    
    Notes
    -----
    Wrapper around find_nearest_idx().
    
    """
    return np.array([vdm.find_nearest_idx(array, val) for val in vals], dtype=int)

In [ ]:
decoded = decode_location(prob, linear)

actual_idx = find_nearest_indices(linear['time'], centers)
actual_location = linear['position'][actual_idx]

decoded[np.isnan(decoded)] = 0
decode_error = np.abs(actual_location - decoded)
np.mean(decode_error)

In [ ]:
print(decoded)

In [ ]:
np.shape(decoded)

In [ ]:
plt.plot(centers, decoded)
plt.plot(linear['time'], linear['position'], 'r.')
plt.show()

In [ ]:
import nengo
smoothed_decoded = nengo.Lowpass(0.002).filtfilt(decoded)

In [ ]:
decode_error = np.abs(actual_location - smoothed_decoded)
print(np.mean(decode_error))

plt.plot(centers, smoothed_decoded)
plt.plot(linear['time'], linear['position'], 'r.')
plt.show()

In [ ]:
for thisposition in linear['time'][10000:10200]:
    x = vdm.find_nearest_idx(pos['time'], thisposition)
    plt.plot(pos['time'][x], pos['x'][x], 'b.')
    plt.plot(pos['time'][x], pos['y'][x], 'g.')

# plt.plot(pos['x'], pos['y'], 'b')
# plt.xlim(linear['time'][10000], linear['time'][12000])
plt.show()

In [ ]:
plt.plot(linear['time'], linear['position'], 'b.')
plt.show()

In [ ]:
centers = edges[:-1] + np.median(np.diff(edges))/2

In [ ]:
print(np.shape(decode['tc'])[1])

In [ ]:
min_neurons = 1
min_spikes = 1

length = np.shape(q)[1]
num_bins = np.shape(decode['tc'])[1]
bin_size = np.median(np.diff(centers))

# occ_uniform = np.ones((1, num_bins))[0] * bin_size



In [ ]:
q.shape

In [ ]:
prob = np.empty((length, num_bins)) * np.nan
for idx in range(num_bins):
    # What does tempprod represent?
    tempprod = np.nansum(np.log(decode['tc'][:,idx][..., np.newaxis] ** q), axis=0)
    
    # What does tempsum represent?
    tempsum = np.exp(-bin_size * np.nansum(decode['tc'][:, idx]))
    
    prob[:, idx] = np.exp(tempprod) * tempsum * (1/num_bins)
    

prob /= np.sum(prob, axis=1)[..., np.newaxis]
# prob[np.isnan(prob)] = 0 add to docstring


In [ ]:
num_active_neurons = np.sum(q > min_spikes, axis=0)
prob[num_active_neurons < min_neurons] = np.nan

In [ ]:
plt.pcolormesh(prob[200::-1])
plt.colorbar()
plt.show()

In [ ]:
max_decoded_idx = np.argmax(prob, axis=1)

In [ ]:
print(np.min(decode['zdata']), np.max(decode['zdata']))
print(0, np.shape(prob)[1]-1)
print(np.min(decoded), np.max(decoded))

In [ ]:
decoded = max_decoded_idx * (np.max(decode['zdata'])-np.min(decode['zdata'])) / (np.shape(prob)[1]-1)
decoded += np.min(decode['zdata'])

In [ ]:
nan_idx = np.sum(np.isnan(prob), axis=1) == (np.shape(prob)[1]-1)

In [ ]:
decoded[nan_idx] = np.nan

In [ ]:
def find_nearest_indices(array, vals):
    return np.array([vdm.find_nearest_idx(array, val) for val in vals], dtype=int)

In [ ]:
rat_location_idx = find_nearest_indices(decode['ztime'], centers)

In [ ]:
np.shape(rat_location_idx)

In [ ]:
rat_location = decode['zdata'][rat_location_idx]

In [ ]:
decode_error = np.abs(rat_location - decoded)

In [ ]:
plt.plot(decode_error)
plt.show()

In [ ]:
np.mean(decode_error)

In [ ]:
plt.plot(centers, decoded)
plt.plot(decode['ztime'], decode['zdata'], 'r.')
plt.show()

In [ ]:
import nengo
smoothed_decoded = nengo.Lowpass(0.001).filtfilt(decoded)

In [ ]:
print(np.mean(np.abs(rat_location - smoothed_decoded)))

In [ ]:
plt.plot(centers, smoothed_decoded)
plt.plot(decode['ztime'], decode['zdata'], 'r.')
plt.show()

In [ ]:
a = np.random.rand(4, 2)
a[:, 1] = np.nan
print(a)
np.argmax(a, axis=0)

In [ ]:
np.sum(np.isnan([np.nan, 2, np.nan]))

In [ ]:
intervals = dict()
intervals['start'] = edges[:-2]
intervals['stop'] = edges[1:-1]

counts = vdm.spike_counts(decode['spikes'], intervals)

In [ ]:
len(intervals['start'])

In [ ]:
print(len(counts), np.min(counts), np.max(counts))